In [8]:
import os, glob, json, joblib, torch, numpy as np, pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import pandas as pd
import numpy as np
import torch
import re
import emoji
from ftfy import fix_text
from datasets import Dataset
from torch.utils.data import DataLoader

from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score, recall_score, f1_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import re, regex
import html, emoji, contractions
from ftfy import fix_text
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

In [3]:
ROOT          = r"D:\University of Illinois Chicago\Classes\CS521\cs521"
BATCH_DIR     = os.path.join(ROOT, "labeled_batches")
MODEL_DIR     = os.path.join(ROOT, "baseline_models")
HINGBERT_DIR  = os.path.join(ROOT, "sarcasm_hingbert_model")
DEVANAGARI_TO_ROMAN = True
LATIN_BLOCK= r'\p{Latin}'
DEVANAGARI_BLOCK = r'\p{Devanagari}'

In [4]:
censored_swear_words = {
    # Handles s!@#, s#!t, s**t, etc., variations of "shit"
    r'\b[Ss][^A-Za-z\s]{3,4}\b': 'shit',
    r'[Ss][\!\@\#\$\%\^\&\*]*[i1\|][\!\@\#\$\%\^\&\*]*[t]+': 'shit',
    r'[Ss][#!@$%^&*+\-=\[\]{};:\'",.<>?/\\|_~`]{2,}[Tt]\b': 'shit',


    # Handles f*ck, f@ck, f**k, f!ck, F*#k, F***, etc., variations of "fuck"
    r'f[\!\@\#\$\%\^\&\*]*[u][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[k]+': 'fuck',
    r'f[\!\@\#\$\%\^\&\*]*[\*]+[c][k]': 'fuck',                # Handles f*ck
    r'f[\!\@\#\$\%\^\&\*]*[\*]+[k]': 'fuck',                   # Handles f***
    r'f[\!\@\#\$\%\^\&\*]*[\*#\!]{1,}[c]?[kz]?': 'fuck',       # Handles F*#k, F***
    r'f[\!\@\#\$\%\^\&\*]*[\w@#$%^&*]+ing\b': 'fucking',        # Hnadles f#%^ing

    # Handles b*tch, b!tch, b1tch, etc., variations of "bitch"
    r'b[\!\@\#\$\%\^\&\*]*[i1\|][\!\@\#\$\%\^\&\*]*[t][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[h]+': 'bitch',

    # Handles "fuck" with various suffixes (e.g., "f***'z" -> "fucks")
    r'f[\!\@\#\$\%\^\&\*]*[u][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[k]+[\'’]?[zs]?': 'fucks',

    # Handles muthaf*ckin, motherf***ing, etc., variations of "motherfucking"
    r'm[ou]*th[\!\@\#\$\%\^\&\*]*[a][\!\@\#\$\%\^\&\*]*f[\!\@\#\$\%\^\&\*]*[u][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[k][\!\@\#\$\%\^\&\*]*[i][\!\@\#\$\%\^\&\*]*[n][\!\@\#\$\%\^\&\*]*': 'motherfucking',
}

In [5]:
hindi_swear_map = {

    # —— “madarchod” ——
    # Roman + common leet/censor + Devanagari
    r'\b(m[a@]d[ae]?[r]?[\W_]*ch[o0]d[a@]?|म[ाअ]*दर[चच]ोद|मादरचोद)\b': 'madarchod',

    # —— “bhenchod / behenchod” ——
    r'\b(b[h]*e+h[e]*n[\W_]*ch[o0]d[a@]?|भेंचोद|बह[ेे]न[चच]ोद|बहनचोद)\b': 'bhenchod',

    # —— “chutiya / ch**iya” ——
    r'\b(c[h]*u+t+i+y*a+|ch[u*]+t+iy?[ae]+|चूतिया|चुतिया|छूतिया)\b': 'chutiya',

    # —— “chu***” shortened form ——
    r'\b(ch[\W_]*u[\W_]*t[\W_]*\*{2,})\b': 'chut***',

    # —— “gandu / gaandu / gaand” ——
    r'\b(g[a@]a*n+d[u]?[aei]?|गां*डू?|गाण्डू?)\b': 'gandu',

    # —— “gaand” stand‑alone  ——
    r'\b(ga+a+nd+|गांड|गांड|गाण्ड)\b': 'gaand',

    # —— “lauda / laudae / laude” —— 
    r'\b(l[a@]u+[d]+[aei]*|लौड़ा|लंड|लुंड)\b': 'lauda',

    # —— “lund” ——
    r'\b(l[u]n+d|लुंड|लंड)\b': 'lund',

    # —— “randi / r@ndi” ——
    r'\b(r[a@]n+d[iy]+|रंडी|रंड़ी)\b': 'randi',

    # —— “harami / haramzada / haramzadi” ——
    r'\b(h[a@]r+a+m[iy]*|हरामी)\b': 'harami',
    r'\b(h[a@]r+a+m[z$]?a+d[a@]|हरामज़ादा|हरामजादा)\b': 'haramzada',
    r'\b(h[a@]r+a+m[z$]?a+d[iy]|हरामज़ादी|हरामजादी)\b': 'haramzadi',

    # —— “kamine / kamina / kaminey” ——
    r'\b(k[a@]m+i+n[e]?[yie]*|कमीना|कमीने|कमीनी)\b': 'kamine',

    # —— “kutta / kutte / kuttiya” ——
    r'\b(k[u]t+t[a@e]*|कुत्ता|कुत्ते|कुते)\b': 'kutta',
    r'\b(k[u]t+t[i]y?a+|कुतिया|कुत्ती)\b': 'kutiya',

    # —— “saala / saale / sali” ——
    r'\b(s+a+a*l+a+|साला|साले)\b': 'saala',
    r'\b(s+a+l+i+|साली)\b': 'sali',

    # —— “bakchod / bakchodi” ——
    r'\b(b[a@]k+ch[o0]d[iy]*|बकचोद|बकचोदी)\b' : 'bakchod',

    # —— milder insults ——
    r'\b(g[a@]dh[a@]a+|गधा|गधे)\b': 'gadha',

    # —— catch‑all “f**k” in Roman Hindi sentences ——
    r'\b(f[\W_]*u[\W_]*c[\W_]*k+)\b': 'fuck',

    # —— emojis or censor blocks containing Devanāgarī letters and stars
    r'[\u0900-\u097F]\*{2,}[\u0900-\u097F]*': 'censored_hindi_word',
}


In [6]:
ascii_emoticons = {
    ':-)': 'smiley_face',
    ':)': 'smiley_face',
    ':-D': 'grinning_face',
    ':D': 'grinning_face',
    ':-(': 'sad_face',
    ':(': 'sad_face',
    ':-P': 'playful_face',
    ':P': 'playful_face',
    ':-p': 'playful_face',
    ':p': 'playful_face',
    ';-)': 'winking_face',
    ';)': 'winking_face',
    ':-O': 'surprised_face',
    ':O': 'surprised_face',
    ':-o': 'surprised_face',
    ':o': 'surprised_face',
    r':-/': 'skeptical_face',
    r':/': 'skeptical_face',
    r':-\|': 'neutral_face',
    r':\|': 'neutral_face',
    '<3': 'heart',
    '^_^': 'happy_face',
    '-_-': 'expressionless_face',
    'o_O': 'confused_face',
    'O_o': 'confused_face',
    'o_o': 'confused_face',
    '>_<': 'frustrated_face',
    'O.o': 'confused_face',
    'o.O': 'confused_face',
    '0_o': 'confused_face',
    'o_0': 'confused_face',
    'xD': 'laughing_face',
    'XD': 'laughing_face',
}


In [7]:
def replace_emojis(text):
    return emoji.demojize(text) if isinstance(text, str) else text

In [9]:
def normalize_text(text):
    if not isinstance(text, str):
        return text
    try:
        # Attempt to encode to 'latin1' and decode to 'utf-8' to fix encoding issues
        clean_text = text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        # Fallback: ignore encoding errors if they occur
        clean_text = text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
    return clean_text

In [10]:
def decode_html_entities(text):
    """Decode HTML entities in text, such as '&amp;' to '&'."""
    if not isinstance(text, str):
        return text
    # Use html.unescape to decode all HTML entities
    clean_text = html.unescape(text)
    return clean_text

In [11]:
def remove_html_tags(text):
    if not isinstance(text, str):
        return text
    clean_text = re.sub(r'</?[^>]+>', '', text)
    return clean_text

In [12]:
def replace_urls(text):
    if not isinstance(text, str):
        return text
    # Regex pattern to match URLs
    url_pattern = r'(http[s]?://\S+|www\.\S+)'
    clean_text = re.sub(url_pattern, ' SomeWebLink ', text)
    return clean_text

In [13]:
def replace_emails(text):
    if not isinstance(text, str):
        return text
    # Pattern to match email addresses
    email_pattern = r'[\w\.-]+@[\w\.-]+\.\w+'
    clean_text = re.sub(email_pattern, ' EMAILADDRESS ', text)
    return clean_text

In [14]:
def replace_censored_swear_words(text):
    if not isinstance(text, str):
        return text
    clean_text = text
    for pattern, replacement in censored_swear_words.items():
        # Do not add word boundaries if pattern already contains them
        if pattern.startswith(r'\b') and pattern.endswith(r'\b'):
            clean_pattern = pattern
        else:
            clean_pattern = r'\b' + pattern + r'\b'
        clean_text = re.sub(clean_pattern, replacement, clean_text, flags=re.IGNORECASE)
    return clean_text

In [15]:
def replace_hindi_swears(text: str) -> str:
    if not isinstance(text, str):
        return text
    out = text
    for pat, repl in hindi_swear_map.items():
        out = regex.sub(pat, repl, out, flags=regex.IGNORECASE)
    return out

In [16]:
def replace_mentions(text):
    """Replace mentions with 'SomeTaggedAccount', including when attached to other words."""
    if not isinstance(text, str):
        return text
    # Pattern to match @ followed by word characters
    # Replace mentions with spaces around to prevent word merging
    mention_pattern = r'@([A-Za-z0-9_]+)'
    clean_text = re.sub(mention_pattern, ' SomeTaggedAccount ', text)
    return clean_text

In [17]:
def replace_at_symbol(text):
    if not isinstance(text, str):
        return text
    # Replace '@' when used as 'at'
    # Conditions:
    # - '@' followed by a space
    # - '@' at the end of a string
    # - '@' followed by a number or time format
    # - '@' followed by a capitalized word (assuming location or time)
    clean_text = re.sub(r'@\s', 'at ', text)
    clean_text = re.sub(r'@$', 'at', clean_text)
    clean_text = re.sub(r'@(?=\d)', 'at ', clean_text)
    clean_text = re.sub(r'@ (?=[A-Z])', 'at ', clean_text)
    return clean_text

In [18]:
def remove_hashtags(text):
    if not isinstance(text, str):
        return text

    # Remove '#' symbol in various contexts
    clean_text = re.sub(r'(^|\s)#(\w*[^\s\w]?)', r'\1\2', text)
    clean_text = re.sub(r'(\w+)#([^\s\w]?)', r'\1\2', clean_text)
    clean_text = re.sub(r'([^\s\w])#', r'\1', clean_text)

    # Remove any remaining standalone '#' symbols
    clean_text = re.sub(r'(?<=\s)#(?=\s|$)|(?<=^)#(?=\s|$)', '', clean_text)

    return clean_text

In [19]:
def expand_contractions(text):
    if not isinstance(text, str):
        return text
    expanded_words = [contractions.fix(word) for word in text.split()]
    clean_text = ' '.join(expanded_words)
    return clean_text

In [20]:
def clean_unicode_punct(text: str) -> str:
    """
    Remove punctuation *except* ? ! . , and keep both Latin & Devanagari letters.
    Any other symbol is dropped.
    """
    if not isinstance(text, str):
        return text
    # keep letters, numbers, spaces and selected punctuation
    pattern = rf'[^\s{LATIN_BLOCK}{DEVANAGARI_BLOCK}0-9\?\!\.,]'
    return regex.sub(pattern, ' ', text)

In [21]:
def transliterate_devanagari(text: str) -> str:
    if not DEVANAGARI_TO_ROMAN or not isinstance(text, str):
        return text
    return transliterate(text, sanscript.DEVANAGARI, sanscript.ITRANS)

In [22]:
def remove_emoji_colons(text):
    """Remove colons around emoji descriptions in the text."""
    if not isinstance(text, str):
        return text
    # Regex pattern to match emoji descriptions with colons
    pattern = r':([a-zA-Z0-9_+-]+):'
    # Replace matches with the emoji description without colons
    clean_text = re.sub(pattern, r'\1', text)
    return clean_text

In [23]:
def replace_ascii_emoticons(text):
    """Replace ASCII emoticons in text with their descriptions."""
    if not isinstance(text, str):
        return text

    clean_text = text
    for emoticon, description in ascii_emoticons.items():
        # Escape the emoticon pattern to handle special characters
        escaped_emoticon = re.escape(emoticon)
        # Build the regex pattern to match emoticons not part of words
        pattern = r'(?<!\w)' + escaped_emoticon + r'(?!\w)'
        # Replace the emoticon with the description, ignoring case
        clean_text = re.sub(pattern, f"{description}", clean_text, flags=re.IGNORECASE)
    return clean_text

In [24]:
def replace_trade_mark(text):
    """Replace 'trade_mark' with a single quote (')."""
    if not isinstance(text, str):
        return text
    # Replace 'trade_mark' with "'"
    clean_text = text.replace("trade_mark", "'")
    return clean_text

In [25]:
def remove_extra_whitespace(text):
    if not isinstance(text, str):
        return text
    clean_text = ' '.join(text.split())
    return clean_text

In [28]:
clean_fns = [
    replace_emojis,
    normalize_text,
    decode_html_entities,
    remove_html_tags,
    replace_urls,
    replace_emails,
    replace_censored_swear_words,
    replace_hindi_swears,
    replace_mentions,
    replace_at_symbol,
    remove_hashtags,
    expand_contractions,
    clean_unicode_punct,
    transliterate_devanagari,
    remove_emoji_colons,
    replace_ascii_emoticons,
    replace_trade_mark,
    remove_extra_whitespace
]

In [29]:
def clean(txt:str) -> str:
    for fn in clean_fns: txt = fn(txt)
    return txt.strip()

In [35]:
LABEL_MAP = {"non_sarcastic": 0, "sarcastic": 1}

def metrics(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    p,r,f,_ = precision_recall_fscore_support(y_true, y_pred, average="binary")
    return acc, p, r, f

In [32]:
# TF-IDF + SVM
tfidf, svm = joblib.load(os.path.join(MODEL_DIR, "tfidf_svm.joblib"))

# BiLSTM
vocab_rows = np.load(os.path.join(MODEL_DIR, "vocab.npy"), allow_pickle=True)
token2idx  = {row[0]: int(row[1]) for row in vocab_rows}   # keys → ints

EMB_DIM = 300
MAXLEN  = 60

class BiLSTM(torch.nn.Module):
    def __init__(self, vocab_size, emb_dim=EMB_DIM, hidden=128):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.lstm  = torch.nn.LSTM(emb_dim, hidden, bidirectional=True, batch_first=True)
        self.fc    = torch.nn.Linear(hidden*2, 2)
    def forward(self, x):
        e,_ = self.lstm(self.embed(x))
        h   = torch.cat([e[:,-1,:128], e[:,0,128:]], dim=-1)  # simple concat last+first dir
        return self.fc(h)

bilstm = BiLSTM(len(vocab)).eval()
state = torch.load(os.path.join(MODEL_DIR, "bilstm_state.pt"), map_location="cpu")
bilstm.load_state_dict(state)

# mBERT baseline
mbert_tok = AutoTokenizer.from_pretrained(os.path.join(MODEL_DIR, "bert_mbert"))
mbert     = AutoModelForSequenceClassification.from_pretrained(
               os.path.join(MODEL_DIR, "bert_mbert")).eval()

# Fine-tuned HingBERT
hing_tok  = AutoTokenizer.from_pretrained(HINGBERT_DIR)
hingbert  = AutoModelForSequenceClassification.from_pretrained(HINGBERT_DIR).eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
bilstm, mbert, hingbert = bilstm.to(device), mbert.to(device), hingbert.to(device)


C:\Users\Manan\AppData\Local\Temp\ipykernel_8660\438787462.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(os.path.join(MODEL_DIR, "bilstm_state.pt")

In [33]:
def bilstm_encode(texts):
    ids = []
    unk = vocab.get("<unk>", 1)
    for t in texts:
        toks = t.split()[:MAXLEN]
        vec  = [vocab.get(tok, unk) for tok in toks] + [0]*(MAXLEN-len(toks))
        ids.append(vec)
    return torch.tensor(ids, dtype=torch.long)

@torch.no_grad()
def predict_all(texts):
    # returns dict of numpy predictions
    cleaned = [clean(t) for t in texts]

    # SVM
    svm_pred = svm.predict(tfidf.transform(cleaned))

    # BiLSTM
    logits   = bilstm(bilstm_encode(cleaned).to(device))
    bil_pred = torch.argmax(logits, 1).cpu().numpy()

    # mBERT
    mb_inp   = mbert_tok(cleaned, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
    mb_pred  = torch.argmax(mbert(**mb_inp).logits, 1).cpu().numpy()

    # HingBERT
    hg_inp   = hing_tok(cleaned, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
    hg_pred  = torch.argmax(hingbert(**hg_inp).logits, 1).cpu().numpy()

    return {"SVM":svm_pred, "BiLSTM":bil_pred, "mBERT":mb_pred, "HingBERT":hg_pred}


In [37]:
results = {m: [] for m in ["SVM", "BiLSTM", "mBERT", "HingBERT"]}

csv_paths = sorted(glob.glob(os.path.join(BATCH_DIR, "labeled_batch_*.csv")))
for path in tqdm(csv_paths, desc="batches"):
    df = pd.read_csv(path).dropna(subset=["sarcasm_label",
                                          "Transliterated_Comment"])

    # 1️ normalise + map to 0/1
    lbl_series = (
        df["sarcasm_label"]
        .astype(str).str.strip().str.lower()
        .map(LABEL_MAP)                      # non_sarcastic → 0, sarcastic → 1
    )

    # 2️ keep only rows that have a valid mapped label
    valid_mask = lbl_series.notna()
    if valid_mask.sum() == 0:
        continue                             # skip empty batch

    y = lbl_series[valid_mask].astype(int).values
    texts = df.loc[valid_mask, "Transliterated_Comment"].astype(str)

    preds = predict_all(texts)

    for m, p in preds.items():
        results[m].append(metrics(y, p))

batches: 100%|██████████| 200/200 [00:18<00:00, 10.92it/s]


In [38]:
summary = {m: np.array(scores).mean(axis=0)
           for m, scores in results.items()}
summary_df = (pd.DataFrame(summary,
                           index=["Accuracy", "Precision", "Recall", "F1"])
              .T.round(4))
summary_df

,Accuracy,Precision,Recall,F1
SVM,0.6356,1.0,0.6356,0.7751
BiLSTM,0.4793,1.0,0.4793,0.6437
mBERT,0.7761,1.0,0.7761,0.8724
HingBERT,0.8735,1.0,0.8735,0.9318
